# Remarques
* 
* 
* 

# Relevant imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from string import digits
import pandas_profiling

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# ignore some warnings 
import warnings
warnings.filterwarnings('ignore')

# Load the data


In [ ]:
#We load the data set into a pandas dataframe
data_to_learn = pd.read_excel('IPO_data_to_learn.xlsx')
data_to_predict = pd.read_excel('IPO_data_to_predict.xlsx')


# Add predictions 



In [ ]:
data_to_learn['P1'] = data_to_learn.closeDay1 > data_to_learn.offerPrice


# Data cleaning

We clean the data to learn from, and we apply the same cleaning to the data to predict

In [ ]:
data_to_learn.columns

In [ ]:
data_to_learn.shape

**Below we see that the Unnamed_0 column represent some kind of indexing on the data which we don't need, hence it makes sence to drop that column.**

In [ ]:
data_to_learn["Unnamed: 0"].unique().shape


In [ ]:
data_to_learn.drop("Unnamed: 0", axis=1, inplace=True)
data_to_predict.drop("Unnamed: 0", axis=1, inplace=True)

**We decided to use pandas_profiling to output a full report of our dataset, we will then proceed to data cleaning accordingly**

In [ ]:
data_to_learn.profile_report(style={'full_width':True})

In [ ]:
profile = data_to_learn.profile_report(title='IPO data Profiling Report')
profile.to_file(output_file="data_profile.html")

## Correlation matrix

In [ ]:
df = data_to_learn.corr()

plt.figure(figsize=(30,30))
ax = sns.heatmap(data=df, xticklabels=df.columns, square=True, annot=True, center=0, cbar=False)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.title('Features Inter-correlation');

## I. Dropping highly correlated rows

We see from the correlation matrix the following interesting correlations:
- `ipoSize` and `totalProceeds` : 1.0
- `amountOnProspectus` and `totalProceeds`: 1.0
- `reputationSum` and `nUnderwriters`: 0.96

We choose to drop `ipoSize`, `amountOnProspectus` and `reputationSum`.

In [ ]:
data_to_learn.drop(["ipoSize", "amountOnProspectus", 'reputationSum'], axis=1, inplace=True)
data_to_predict.drop(["ipoSize", "amountOnProspectus", 'reputationSum'], axis=1, inplace=True)

## II. Dealing with missing values

**Depending on the distribution of the variable, we replace missing values using the Mean or Variance: If the distribution is skewed or heavy tailed we use the median because it's more robust to outliers**

In [ ]:
#For feature city we dropped the only record which had NaN because it also had NaN in other columns
data_to_learn.dropna(subset=['city'], inplace = True)
data_to_predict.dropna(subset=['city'], inplace = True)

In [ ]:
# It made sence to give the value 0 to the missing values in leverage as it's the most common value 
#[https://www.thebalancesmb.com/how-leverage-can-benefit-your-business-398312]
data_to_learn['leverage'].fillna(value=0, inplace=True)
data_to_predict['leverage'].fillna(value=0, inplace=True)

In [ ]:
#For feature blueSky, we replaced missing values by th evalue zero, 
#because in some states of the US there are no Blue Sky expenses.
#[https://www.colonialstock.com/blue-sky-state-filing-fees.htm]
data_to_learn['blueSky'].fillna(value=0, inplace=True)
data_to_predict['blueSky'].fillna(value=0, inplace=True)

In [ ]:
#For features age, bookValue, investmentReceived, managementFee we used the median
values = {'age': data_to_learn["age"].median(),\
          'bookValue': data_to_learn["bookValue"].median(),\
          'investmentReceived': data_to_learn["investmentReceived"].median(),\
          'managementFee': data_to_learn["managementFee"].median(),\
          'commonEquity': data_to_learn["commonEquity"].median(),\
          'commonEquity.1': data_to_learn["commonEquity.1"].median(),\
          'investmentReceived': data_to_learn["investmentReceived"].median(),\
          'nExecutives': data_to_learn["nExecutives"].median(),\
          'netIncome': data_to_learn["netIncome"].median(),\
          'nVCs': data_to_learn["nVCs"].median(),\
          'patRatio': data_to_learn["patRatio"].median(),\
          'priorFinancing': data_to_learn["priorFinancing"].median(),\
          'roa': data_to_learn["roa"].median(),\
          'sharesOfferedPerc': data_to_learn["sharesOfferedPerc"].median(),\
          'totalAssets': data_to_learn["totalAssets"].median(),\
          'totalRevenue': data_to_learn["totalRevenue"].median()
         }

data_to_learn.fillna(value=values, inplace=True)
data_to_predict.fillna(value=values, inplace=True)

In [ ]:
## closeDay1 is what we want to predict
## We drop the NaN values since it is akin to non-label data and they only represent 3% of the data

print("closeDay1 == NaN: ", data_to_learn.closeDay1.isna().sum())
print("closeDay1 == NaN: ", data_to_learn.closeDay1.isna().sum()/data_to_learn.closeDay1.shape[0]*100, '%')

data_to_learn = data_to_learn.dropna(subset=['closeDay1'])

In [ ]:
# replace NaN rf by empty string 

data_to_learn.fillna('', inplace=True)
data_to_predict.fillna('', inplace=True)

## II. Data Cleaning

First let's give our columns more meaningful names

In [ ]:
data_to_learn.rename(columns={
    "commonEquity": "commonEquityPercentage",\
    "commonEquity.1": "commonEquity"
}, inplace=True)

### patRatio

We get rid of infinite values and set them to 0 since they correspond to (occurences of "patent" in the rf)/(length of rf)


In [ ]:
data_to_learn.patRatio[data_to_learn.patRatio == np.inf].count()

data_to_learn.patRatio.replace(np.inf, 0.0, inplace=True)
data_to_predict.patRatio.replace(np.inf, 0.0, inplace=True)

### City
We clean the City feature: we remove numbers to avoid duplicates and leading and trailing whitespaces

In [ ]:
# We remove numbers to avoid duplicates and leading and trailing whitespaces
data_to_learn["city"] = data_to_learn["city"].str.replace('\d+', '').str.strip()
data_to_predict["city"] = data_to_predict["city"].str.replace('\d+', '').str.strip()


### industryFF5, industryFF12, industryFF48

They are industry classifications with respectivelly 5, 12 and 48 categories.

We will label encode them 

In [ ]:
data_to_learn.industryFF5.value_counts().shape

In [ ]:
data_to_learn.industryFF12.value_counts().shape

In [ ]:
data_to_learn.industryFF48.value_counts().shape

In [ ]:
# Hot-encode
data_to_learn = pd.get_dummies(data=data_to_learn, columns=['industryFF5', 'industryFF12', 'industryFF48'])
data_to_predict = pd.get_dummies(data=data_to_predict, columns=['industryFF5', 'industryFF12', 'industryFF48'])

### Exchange

The exchange is either `NASDQ`, `NYSE` or `ÀMEX`. 

We hot-encode it

In [ ]:
data_to_learn.exchange.value_counts()

In [ ]:
# Hot-encode
data_to_learn = pd.get_dummies(data=data_to_learn, columns=['exchange'])
data_to_predict = pd.get_dummies(data=data_to_predict, columns=['exchange'])

### Issuer

`issuer` is unique. We assume it therefore gives no predictive value and we drop it.

In [ ]:
data_to_learn.issuer.is_unique

In [ ]:
data_to_learn.drop(["issuer"], axis=1, inplace=True)
data_to_predict.drop(["issuer"], axis=1, inplace=True)

### Process `manager`

#### Preprocessing

- Separate the managers (The managers are separated by `\n`)

#### Feature engineering

- Add columns with the amount of managers (it might be useful)
- Compute the average of the success rates of each manager


In [ ]:
data_to_learn.manager.head()


In [ ]:
data_to_learn.reset_index(drop=True, inplace=True)
data_to_predict.reset_index(drop=True, inplace=True)

In [ ]:
data_to_learn.manager = data_to_learn.manager.apply(lambda x: set(x.split('\n')))
data_to_predict.manager = data_to_predict.manager.apply(lambda x: set(x.split('\n')))

In [ ]:
data_to_learn['managerLength'] = data_to_learn.manager.apply(lambda x: len(x))
data_to_predict['managerLength'] = data_to_predict.manager.apply(lambda x: len(x))

In [ ]:
# List the index where the IPO is successful
success = data_to_learn.closeDay1 > data_to_learn.offerPrice
successful_index = success.index.to_list()

In [ ]:
# Compute the average of the success rates of each manager
for i, managers in enumerate(data_to_learn.manager):
    average_success_rates = []
    for manager in managers:
        average_success_rate = success[data_to_learn.manager.apply(lambda x: manager in x)].mean()
        average_success_rates.append(average_success_rate)
    data_to_learn.loc[i, 'managerSuccessAvg'] = np.array(average_success_rates).mean()
        
        
for i, managers in enumerate(data_to_predict.manager):
    average_success_rates = []
    for manager in managers:
        average_success_rate = success[data_to_learn.manager.apply(lambda x: manager in x)].mean()
        average_success_rates.append(average_success_rate)
    data_to_predict.loc[i, 'managerSuccessAvg'] = np.array(average_success_rates).mean()

In [ ]:
data_to_learn.drop('manager', axis=1, inplace=True)
data_to_predict.drop('manager', axis=1, inplace=True)

### Process `rf`

#### Preprocessing

- Clean punctuation
- Remove stopwords
- Lemmatize
- Lowercase

#### Feature engineering

- Store `rf` length, it might be useful
- Create scores relating the similarity between rf entries
    - Create a corpus where each document is a `rf` entry (1)
    - Compute the TF-IDF representation of each of those documents
    - Compute the **cosine-similarity** between **each** document's TF-IDF representation and **all** the TF-IDF representations of the **successful** IPO
    - Store the success rate of the top 1, 10 and 100 similarity rf

    When a new document appears:
    - Compute the TF-IDF of this new document (using (1) as corpus)
    - Compute the **cosine-similarity** of your new document's TF-IDF representation and **all** the TF-IDF representations of the **successful** IPO
    - Store the success rate of the top 1, 10 and 100 similarity rf


In [ ]:
df = data_to_learn.rf.apply(lambda x: len(x.split()))
df.hist(bins=100)
print("Mean rf word length: ", df.mean())
print("Std rf word length: ", df.std())
plt.title("Histogram of the risk factor's word length")
plt.xlabel("Risk factor's word length")
plt.ylabel("Occurences")

In [ ]:
import re 
import spacy
import nltk
from nltk import corpus

stopwords = set(corpus.stopwords.words('english'))

def clean_re(txt):
    return re.sub(r'[^\w\s]', '', txt) # We get rid of (non-words, non-whitespaces)

def remove_stopwords(txt):
    removed_txt = []
    for token in txt.split():
        if not token in stopwords:
            removed_txt.append(token)
    return ' '.join(removed_txt)

def lemmatize_text(txt):
    wnl = nltk.stem.WordNetLemmatizer() # maybe use a better lemmatizer eventually 
    
    lemmatized_txt = []
    for token in txt.split():
        lemmatized_txt.append(wnl.lemmatize(token))
    return ' '.join(lemmatized_txt).strip()

def preprocess_text(txt):
    txt = clean_re(txt)
    txt = remove_stopwords(txt.lower())
    txt = lemmatize_text(txt)
    return txt

In [ ]:
data_to_learn.rf = data_to_learn.rf.apply(preprocess_text)
data_to_predict.rf = data_to_predict.rf.apply(preprocess_text)


In [ ]:
data_to_learn['rfLength'] = data_to_learn.rf.apply(lambda x: len(x.split()))

In [ ]:
# List the index where the IPO is successful
success = data_to_learn.closeDay1 > data_to_learn.offerPrice
successful_index = success.index.to_list()

In [ ]:
# Compute the TF-IDF score of each rf entry
tf_idf_vectorizer = TfidfVectorizer(ngram_range=(1,3), min_df=0)
tf_idf_vectorizer.fit(data_to_learn.rf)
tf_idf_matrix = tf_idf_vectorizer.transform(data_to_learn.rf)

In [ ]:
# Compute the **average cosine-similarity** between **each** document's TF-IDF representation and top1, top10 and top100 of the TF-IDF representations of the **successful** IPO
for i, _ in enumerate(data_to_learn.rf):
    cosine_similarities = linear_kernel(tf_idf_matrix[i], tf_idf_matrix[successful_index]).flatten()
    top_cosine_similarities_index = [j for j in cosine_similarities.argsort()[::-1] if i != j]
    data_to_learn.loc[i, 'rfSuccessAvgTop1'] = success[top_cosine_similarities_index[0:1]].mean()
    data_to_learn.loc[i, 'rfSuccessAvgTop10'] = success[top_cosine_similarities_index[0:10]].mean()
    data_to_learn.loc[i, 'rfSuccessAvgTop100'] = success[top_cosine_similarities_index[0:100]].mean()

In [ ]:
# Compute the **average cosine-similarity** between **each** document's TF-IDF representation and top1, top10 and top100 of the TF-IDF representations of the **successful** IPO
for i, rf in enumerate(data_to_predict.rf):
    cosine_similarities = linear_kernel(tf_idf_vectorizer.transform(rf), tf_idf_matrix[successful_index]).flatten()
    top_cosine_similarities_index = [j for j in cosine_similarities.argsort()[::-1] if i != j]
    data_to_predict.loc[i, 'rfSuccessAvgTop1'] = success[top_cosine_similarities_index[0:1]].mean()
    data_to_predict.loc[i, 'rfSuccessAvgTop10'] = success[top_cosine_similarities_index[0:10]].mean()
    data_to_predict.loc[i, 'rfSuccessAvgTop100'] = success[top_cosine_similarities_index[0:100]].mean()

In [ ]:
data_to_learn.drop('rf', axis=1, inplace=True)
data_to_predict.drop('rf', axis=1, inplace=True)

In [ ]:
data_to_learn.to_pickle('data_to_learn.pkl')
data_to_predict.to_pickle('data_to_predict.pkl')

In [ ]:
data_to_learn = pd.read_pickle('data_to_learn.pkl')
data_to_predict = pd.read_pickle('data_to_predict.pkl')

In [ ]:
df = data_to_learn[0:100].corr()

plt.figure(figsize=(30,30))
sns.heatmap(data=df, xticklabels=df.columns, square=True, annot=True, center=0)

## III. Dealing with outliers

In [ ]:
floatCols = data_to_learn.loc[:, data_to_learn.dtypes == float].columns

In [ ]:
outliers = []
for col in floatCols:
    mean = data_to_learn[col].mean()
    std = data_to_learn[col].std()
    for i,value in enumerate(data_to_learn[col]):
        if value < mean-7*std or value > mean+7*std:
            print('Outlier in', col, 'at index', i)
            outliers.append([col, i])

In [ ]:
print(data_to_learn['nExecutives'].iloc[60:64]) #ça a l'air suspect par exemple...

# Exploratory Data Analysis

# ML


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.dummy           import DummyClassifier
from sklearn.linear_model    import LogisticRegression
from sklearn.svm             import SVC
from sklearn.neighbors       import KNeighborsClassifier
from sklearn.tree            import DecisionTreeClassifier
from sklearn.ensemble        import RandomForestClassifier
from sklearn.ensemble        import GradientBoostingClassifier

from sklearn.pipeline        import Pipeline
from sklearn.metrics         import confusion_matrix
from sklearn.metrics         import roc_auc_score
from sklearn.metrics         import roc_curve


SEED = 1

In [ ]:
prediction_labels = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9']
X_to_predict = data_to_predict[list(set(data_to_predict.columns) - set(prediction_labels))]


## Prediction P1

- Predict whether the **closing price at the end of the first day** (i.e. `closeDay1`) of trading will go up (the "positive" case, coded as 1) or down (the "negative" case, coded as 0) from the **offer price** `offerPrice`. You may use all data from the dataset except for the rf variable (i.e., risk factors).


### Preliminary

We want to predict whether the offer price is **underpriced** or **overpriced**.

The **label to predict** is **"Is offer price underpriced ?"**. 

Therefore, 
- **False Positives** are especially **costly**, (we would actually lose money) 
- **False Negatives are not costly** (we would simply miss an opportunity to make money) 
- **True Positives** make us money 
- **True Negatives** avoid us losing money.

Let's use the AUC

In [ ]:
# Prepare the dataset to learn from
y = data_to_learn['closeDay1'] > data_to_learn['offerPrice']

X = data_to_learn.drop(['closeDay1', 'rfSuccessAvgTop1', 'rfSuccessAvgTop10', 'rfSuccessAvgTop100'], axis=1) # We remove the features engineered from rf

# For now, we discard the city, the managers and issuer. Later, hot-encode or something
X.drop(['city', 'manager', 'issuer'], axis=1, inplace=True)


## Split into test, train, validation sets

From the data to train from, 20% is test set, 20% is train set and 20% is test set

In [ ]:
# We start by extracting our test set (20% of all data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=SEED, stratify=y)

# We then divide our initial training set into our actual training set and our validation set (20%)
X_train_train, X_train_val, y_train_train, y_train_val  = train_test_split(X_train, y_train, test_size=0.25, random_state=SEED, stratify=y_train)

### KNN

In [ ]:
# Build pipeline
steps = []
steps.append(('standardize', StandardScaler()))
steps.append(('knn_model', KNeighborsClassifier()))
pipeline = Pipeline(steps)

In [ ]:
# Tune K
results = []
for k in range(1, 100, 5):
    pipeline.set_params(knn_model__n_neighbors=k) 
    pipeline.fit(X_train_train, y_train_train)
    y_train_val_hat_prob = pipeline.predict_proba(X_train_val)[:,1]
    auc = roc_auc_score(y_train_val, y_train_val_hat_prob)
    results.append((auc, k))

# View results 
results = pd.DataFrame(results)
results.columns = ['AUC', 'K']
sns.lineplot(x='K', y='AUC', data=results)
plt.axhline(y=0.5, c='r', linestyle='--', label='Random')
plt.legend()
plt.title('AUC score compared for KNN for different K')
print('Max AUC with K: ', results[results.AUC == results.AUC.max()].K)

### Random Forest

In [ ]:
# Build pipeline
steps = []
steps.append(('forest_model', RandomForestClassifier(random_state=SEED)))
pipeline = Pipeline(steps)

In [ ]:
# Tune N   
results = []
for n in range(1, 100, 5):
    pipeline.set_params(forest_model__n_estimators = n) 
    pipeline.fit(X_train_train, y_train_train)
    y_train_pred = pipeline.predict_proba(X_train_val)
    auc = roc_auc_score(y_train_val, y_train_pred[:,1])
    results.append( (auc, n))

# View results 
results = pd.DataFrame(results)
results.columns = ['AUC', 'N']
sns.lineplot(x='N', y='AUC', data=results)
plt.axhline(y=0.5, c='r', linestyle='--', label='Random')
plt.legend()
plt.title('AUC score compared for Random Forest for different N')
print('Max AUC with N: ', results[results.AUC == results.AUC.max()].N)

## Logit

In [ ]:
steps = []
steps.append(('standardize', StandardScaler()))                            # standardize features
steps.append(('logit_model_l1', LogisticRegression(random_state=SEED)))    # use a logit model
logit_pipeline = Pipeline(steps)
logit_pipeline.set_params(logit_model_l1__penalty='l1')

In [ ]:
results = []
for c in np.linspace(1, 400, 401):
    logit_pipeline.set_params(logit_model_l1__C=c) 
    logit_pipeline.fit(X_train_train,y_train_train)
    y_train_pred = logit_pipeline.predict_proba(X_train_val)       # use validation set during hyper-parameter tuning
    auc_lml1 = roc_auc_score(y_train_val, y_train_pred[:,1])   
    results.append([auc_lml1, c])

In [ ]:
plt.figure(figsize=(10,5))
plt.plot([row[1] for row in results], [row[0] for row in results])
plt.xlabel('C')
plt.ylabel('AUC')
plt.title('Optimizing for C');
plt.grid()

## Prediction P2

- Predict whether the **closing price at the end of the first day** of trading will go up (the "positive" case, coded as 1) or down (the "negative" case, coded as 0) from the **offer price.** You may use only the **rf** (i.e., risk factors), **year**, and **industryFF12** variables for this prediction task. You may, however, perform additional text analysis of the rf variable.


In [ ]:
# Prepare the dataset to learn from
y = data_to_learn['closeDay1'] > data_to_learn['offerPrice']


X = data_to_learn[['rfSuccessAvgTop1', 
                   'rfSuccessAvgTop10',
                   'rfSuccessAvgTop100',
                   'year'] + [industryFF12_column for industryFF12_column in data_to_learn.columns if industryFF12_column.startswith('industry')]]

In [ ]:
X.head()

## Prediction P3

- Predict whether the closing price at the end of the first day of trading will go up (the "positive" case, coded as 1) or down (the "negative" case, coded as 0) from the offer price.

## Prediction P4

- Predict whether the closing price at the end of the first day of trading will go up by more than 20% from the original offer price (the "positive" case, coded as 1) or not (the "negative" case, coded as 0).


## Prediction P5

- Predict whether the closing price at the end of the first day of trading will go down by more than 20% from the original offer price (the "positive" case, coded as 1) or not (the "negative" case, coded as 0).

## Prediction P6

- Predict the share price at the end of the first day.

## Prediction P7

- Predict the probability that the closing price at the end of the first day of trading will go up by more than 5% from the original offer price.

## Prediction P8

- Predict the probability that the closing price at the end of the first day of trading will go up by more than 50% from the original offer price.

## Prediction P9

- Predict the probability that the closing price at the end of the first day of trading will go down (the "positive" case, coded as 1) or not (coded as 0) by more than 10% from the original offer price.